# Custom Agent with Langchain and Comet

This section will cover all the details of creating a custom agent using Langchain and Comet. The agent will be responsible for answering queries related to medical questions. Later, we will employ Comet to monitor the progress of our agent.



# Setup & Imports

To start, we require some packages and libraries. This section will set up all the required ones for the demo. Signing up to Comet is necessary, in case you haven't already done so. For that, just go to [Comet](https://www.comet.com/) and sign up. After you are in the Comet space(dashboard), go to **API keys** by clicking your profile as shown below:

![1.png](images/1.png)

After this, create an API key if not already done so. It's shown below:

![2.png](images/2.png)

In [1]:
!pip -q install langchain tiktoken duckduckgo-search comet_llm openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import re
import langchain
from typing import List, Union
from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser

In [3]:
import comet_llm

PROJECT_NAME = "medical-agent-v1"
# initialize comet
comet_llm.init(project=PROJECT_NAME) # will ask for Comet API

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


# Tools 🧰

Tools serve as interfaces for agents, chains, or LLMs to engage with the environment. It comprises of the tool name, its description, JSON schema for inputs, function invocation, and the option to directly return results to users.

In the following code snippet, we'll craft a straightforward tool using `DuckDuckGoSearchRun` to conduct internet searches for our queries. Subsequently, the real-time nature of the query results will become evident.

In [4]:
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="Handy when you're required to respond to inquiries regarding current event."
    )
]

search.run("What is the behaviour of stock market today?")

'Behavioral finance is a field of finance that proposes psychology-based theories to explain stock market anomalies such as severe rises or falls in stock price. Within behavioral finance , it is ... The stock market presents investors with many data points that can help them make well-informed, logical decisions. While we have the resources to make logical decisions, emotions drive most of ... Stock market psychology refers to the study of emotions, behaviors, and mental processes of investors impacting their trading decisions and market behavior. It is crucial for investors and analysts as it helps explain market movements, the formation of trends, and the occurrence of market anomalies. You are free to use this image on your ... Market sentiment is the overall attitude of investors toward a particular security or financial market . Market sentiment is the feeling or tone of a market, or its crowd psychology, as revealed ... Measures of market volatility have plunged to levels last s

However, for creating a medical agent, relying solely on a search engine might not suffice. Therefore, we ensure that the search engine specifically accesses [WebMD](https://www.webmd.com/), known for its reliability in medical information. Here's how to do it:

In [5]:
search.run("site:webmd.com What's the treatment for Tuberculosis?")

"Tuberculosis (TB) is a bacterial infection that can be dangerous, but it's almost always curable. ... What's the Treatment for Tuberculosis? Medically Reviewed by Paul Boyce, MD on November 03 ... Tuberculosis is a serious infectious disease that affects the lungs. Learn about its causes, symptoms, and treatment options in this comprehensive guide. Treatments to reverse the damage of bone tuberculosis include:\u200c Anti-tuberculosis drugs. Rifampicin, streptomycin, kanamycin, isoniazid, protionamide, cycloserine, and pyrazinamide are the most ... TB Skin Testing Results. If you have a raised, hard bump or there's swelling on your arm, you have a positive test. That means TB germs are in your body. But it doesn't always mean you have active ... 4 min read. Scrofula, also called cervical tuberculous lymphadenitis, is a type of tuberculosis infection. It's caused by the same bacteria that causes pulmonary tuberculosis (TB). Tuberculosis is ..."

Here's how to create a tool out of it:

In [6]:
def serach_md(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools.append(Tool(
        name = "Search WebMD",
        func=serach_md,
        description="Handy when you're required to answer medical and pharmalogical questions"
    )
)

# Prompt

Prompts tell agents what to do and to pass it to the agent it mush have the following components:

1. **Tools**: The agent should be aware of the tools at its disposal.

2. **Intermediate Steps**: These are comprised of previous pairs of `Agent Actions` and `Observations`. Although not directly provided to the model, the prompt template structures them in a particular manner.

3. **Input**: This refers to the generic input provided by the user.

The agent utilizes these components to effectively carry out its tasks and provide accurate and helpful responses to the user's input.

The prompt in the code snippet is working like an action and reason engine. In technical terms, it is referred to as ReACT (Reason + Action). ReAct combines LLM's reasoning and its acting capabilities as seen below:

![3.png](images/3.png)

In [7]:
# Base template

template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""


In the code snippet below, we are taking the `intermediate_steps` that comprise `action` and `observation` as a tuple. We will add it to `agent_scratchpad` so it keeps track of previous actions and observations so the agent knows what work it has already done.

To make the work easier, we will inherit the class from `StringPromptTemplate` will override the `format` function and set up additional parameters like `agent_scratchpad`, `tools`, and `tool_names` and call the parent class `format` function

In [8]:
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [9]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [10]:
prompt

CustomPromptTemplate(input_variables=['input', 'intermediate_steps'], template='Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Remember to answer as a compansionate medical professional when giving your final answer.\n\nQuestion: {input}\n{agent_scratchpad}', tools=[Tool(name='Search', description="Handy when you're required to respond to inquiries regarding current event.", func=<bound method BaseTool.run of DuckDuckGoSearchRun()>), Tool(name=

# Output Parsers

Output parsers transform the output of the LLM to some other suitable format like JSON. In the case of agents, the output parser transforms the agent output into either AgentAction or AgentFinish.

1. **AgentAction**: This data class represents the action to be executed by an agent. Its properties are the name of the tool to invoke (`tool`) and its corresponding input (`tool_input`).

2. **AgentFinish**: It's the final result from the agent. It contains a key-value mapping named `return_values`, housing the final agent output. Typically, this output contains a string response under the output key.

In the code snippet below, we will check if the agent has a final answer. If it does, we will return the result wrapped in the `AgentFinish` object. Otherwise, we will pass the action and action input to the `AgentAction` function so the agent can execute the action.

In [11]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`") # If the template is not maintained it will throw this
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [12]:
output_parser = CustomOutputParser()

In [13]:
output_parser.parse("Action:webmd \nAction Input: Treatment for Tuberculosis") # Returns AgentAction
#output_parser.parse("Webmd Treatment for Tuberculosis") # Value Error
output_parser.parse("Final Answer: Tuberculosis (TB) is a bacterial infection that can be dangerous...") # Returns AgentFinish

AgentFinish(return_values={'output': 'Tuberculosis (TB) is a bacterial infection that can be dangerous...'}, log='Final Answer: Tuberculosis (TB) is a bacterial infection that can be dangerous...')

# Chains and LLMs

Large Language Models (LLMs) are a fundamental part of LangChain and the computational engine for an agent is a LLM. Their ability to reason stems from their training on vast amounts of textual data. It's plausible that human reasoning can be articulated in text, thus allowing LLMs to learn and replicate this cognitive process.

Now, we will create a chain (sequences of calls, whether to an LLM, a tool, or a data preprocessing step) comprising of LLM and the above prompt.

In [14]:
llm = OpenAI(temperature=0, openai_api_key="...")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [15]:
llm_chain = LLMChain(llm=llm, prompt=prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [16]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMSingleActionAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


# Agent Executor

The agent executor serves as the runtime for an agent. It initiates the agent, executes its chosen actions, returns the action outputs, and repeats the process.


In [17]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,
                                                    tools=tools,
                                                    verbose=True)

In [18]:
agent_executor.invoke("I have been bitten by a snake!! What is the first thing to do?")



> Entering new AgentExecutor chain...
Thought: Remain calm and assess the situation.
Action: Search
Action Input: "snake bite first aid"

Observation:Learn how to handle a venomous snakebite and when to seek medical help. Find out the symptoms, types and locations of venomous snakes in North America and how to avoid them. Learn how to handle snake bites, whether venomous or not, and when to call 911. Find out what to do before, during and after the bite, and how to prevent infections and complications. Learn how to treat snake bites as a medical emergency, no matter the type of snake or the severity of the bite. Find out the symptoms, causes, diagnosis and treatment options for venomous and dry snake bites, as well as how to prevent and live with snake bites. To help someone with a snakebite: Take off all jewelry and tight clothing to avoid problems with swelling. Keep the area of the bite below the heart to keep venom from spreading. Keep the person ... Venomous snake bites: symptom

{'input': 'I have been bitten by a snake!! What is the first thing to do?',
 'output': 'The first thing to do after being bitten by a snake is to remain calm and assess the situation. Then, search for information on how to handle a venomous snakebite and when to seek medical help. Follow the instructions for first aid, such as removing jewelry and tight clothing, keeping the area of the bite below the heart, and seeking medical attention immediately.'}

As of now, we are done with the coding part related to an agent. In the next section, we will monitor the question answering using Comet. Let's get into it.

# Creating a Project in Comet

To create a project in Comet space, On Comet's dashboard click on **New Project** as shown in the image:

![4.png](images/4.png)

Next, simply name the project and make sure it matches with the `PROJECT_NAME` defined in the `comet_llm.init()`. Add an optional description and select **Large Language Models (Tailored visualizations for analyzing
prompts and chains)** as project type. Here is the setup:

![5.png](images/5.png)

# Trace and Monitor on Comet ☄

To monitor and track Comet, you can utilize the `CometTracer` function. This function helps you track the outputs of the agent and assess if the prompt is suitable for your use case.

Tracing the agent can be easily accomplished. We can use `CometTracer` as a callback, and all the traces will then appear on the Comet dashboard.

In [22]:
from langchain.callbacks.tracers.comet import CometTracer


queries = ["How can I treat a headache?",
           "How is tuberclousis caused?",
           "I have been bitten by a snake!! What is the first thing to do?"]

expected_response = [
    "Rest in a quiet, dark room. Use hot or cold compresses on your head or neck. Consider a gentle massage and small amounts of caffeine.",
    "Tuberculosis is caused by a bacterial infection called Mycobacterium tuberculosis. It primarily affects the lungs but can also spread to other body parts. Seeking treatment promptly is crucial to prevent the disease from spreading and to ensure a full recovery.",
    "If bitten by a snake, stay calm and assess the situation. Remove any jewelry or tight clothing, keep the bite area below the heart level, and seek immediate medical attention.",
]

callback = CometTracer()

for query in queries:

  agent_executor.run(query,callbacks=[callback])




> Entering new AgentExecutor chain...
Thought: Headaches can be caused by a variety of factors, so it's important to gather more information before recommending a treatment.
Action: Search WebMD
Action Input: "headache treatment"

Observation:Migraine headaches, also known as migraines, are a type of headache that can cause debilitating pain. Learn more about the types, causes, symptoms, diagnosis, & treatment of migraines. Treatment. For tension headaches and migraines, o ver-the-counter (OTC) pain relief with ingredients such as acetaminophen, aspirin, or ibuprofen can help. Repeated or severe migraines and cluster ... A combo of acetaminophen + aspirin + caffeine. Tell your doctor if OTC drugs don't help or you use them a lot. Many people with migraine often need prescription treatment to feel better. And ... Primary Headache. This means the headache itself is the main medical condition. This includes: Migraines. Tension-type headaches. Cluster headaches. New daily persistent head

As we run the above code snippet, the trace will be logged to the Comet under the provided project name as shown below:

![6.png](images/6.png)

As you see above, the input and output logs are stored with respective names. It also logs the duration for each query. We can further investigate by clicking the rows. This will open up a chain trace as seen below:

![7.png](images/7.png)

As shown above, the process begins by providing input to `ChatOpenAI` to identify the necessary tool. Once identified, it conducts the relevant search, passing the results as context to the chain, and ultimately provides an answer.

Additionally, it includes supplementary information such as the execution time for each step and corresponding metadata.